####Run the below code to start real-time emotion detection.

Note: Don't forget to change the filepaths accordingly.


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import numpy as np
from PIL import Image
import io
import cv2
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import imutils
import numpy as np
def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);

      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg');
            pendingResolve(result);
        })
    }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }

  ''')
  display(js)

def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def image2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x

VideoCapture()
eval_js('create()')
face_detector = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')
emotion_classifier = load_model("/content/_mini_XCEPTION.80-0.66.hdf5", compile=False)
EMOTIONS = ["Angry","Disgust","Scared", "Happy", "Sad", "Surprised","Neutral"]


while True:
  byte = eval_js('capture()')
  im = byte2image(byte)
  #print(im.shape)
  gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
  faces = face_detector.detectMultiScale(gray, scaleFactor=1.1,minNeighbors=5,minSize=(30,30),)
  faces = sorted(faces, reverse=True,key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))
  if len(faces) > 0:
    for face_coordinates in faces:
        (fX, fY, fW, fH) = face_coordinates
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]
        if label == "Angry":
            cv2.putText(im, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 0, 0), 1)
            cv2.rectangle(im, (fX, fY), (fX + fW, fY + fH),(255, 0, 0), 1)
        elif label == "Sad":
            cv2.putText(im, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 1)
            cv2.rectangle(im, (fX, fY), (fX + fW, fY + fH),(0, 0, 255), 1)
        elif label == "Happy":
            cv2.putText(im, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 0), 1)
            cv2.rectangle(im, (fX, fY), (fX + fW, fY + fH),(255, 255, 0), 1)
        elif label == "Surprised":
            cv2.putText(im, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 255), 1)
            cv2.rectangle(im, (fX, fY), (fX + fW, fY + fH),(0, 255, 255), 1)
        elif label == "Neutral":
            cv2.putText(im, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 1)
            cv2.rectangle(im, (fX, fY), (fX + fW, fY + fH),(255, 255, 255), 1)
        elif label == "Scared":
            cv2.putText(im, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,140,0), 1)
            cv2.rectangle(im, (fX, fY), (fX + fW, fY + fH),(255,140,0), 1)
        else:
            cv2.putText(im, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 1)
            cv2.rectangle(im, (fX, fY), (fX + fW, fY + fH),(0, 255, 0), 1)
    eval_js('showimg("{}")'.format(image2byte(im)))
cv2.destroyAllWindows()